In [ ]:
# OBJETIVO DO NOTEBOOK
# 1º Normalizar dados númericos e categóricos
# 2º Caslibrar o número de grupos para o cluster (Otimização)
# 3º Treinar o modelo de cluster 
# 4º Descrever os grupos de modelo de cluster (Centróides)
# 5º Definir a qual grupo uma nova instância pertence

###########################################
# TRABALHO DE NORMALIZAÇÃO
# Data Since: 07/04/2023
# @Autor: Santian

!python.exe -m pip install --upgrade pip
!pip install pandas==1.5.2
!pip install matplotlib
!pip install scikit-learn

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing 
from pickle import dump, load
scaler = StandardScaler()

ABRE O ARQUIVO E REALIZA A LEITURA DO MESMO

In [ ]:
# LE O ARQUIVO
dataframe = pd.read_csv('codon_usage.csv\codon_usage.csv', sep=',')
# IMPRIME AS 5 PRIMEIRAS LINHAS
dataframe

In [ ]:
# Verifica quais colunas são categóricas
colunas_categoricas = [coluna for coluna in dataframe.columns if dataframe[coluna].dtype == 'object']

colunas_categoricas

In [ ]:
normalizador = preprocessing.MinMaxScaler()

# converte as colunas 'UUU' e 'UUC' para numérico, com valores inválidos convertidos para NaN
dataframe['UUU'] = pd.to_numeric(dataframe['UUU'], errors='coerce')
dataframe['UUC'] = pd.to_numeric(dataframe['UUC'], errors='coerce')

# preenche valores faltantes com a média para colunas numéricas
df_num = dataframe.select_dtypes(include=['float64', 'int64'])
df_num = df_num.fillna(df_num.mean())


# Obter o modelo normalizador e salvar para uso posterior
modelo_normalizador = normalizador.fit(df_num) # O método fit é responsável por obter o modelo que normalizará a base 
dados_num_normalizado = normalizador.fit_transform(df_num) # Responsável por aplicar a normalização

#alternativa
dados_num_normalizado = modelo_normalizador.fit_transform(df_num)

#Salvar o modelo normalizador em disco
dump(modelo_normalizador,open('normalizador1.model','wb'))

# preenche valores faltantes com a moda para colunas categóricas
df_cat = dataframe.select_dtypes(include=['object'])
df_cat = df_cat.fillna(df_cat.mode().iloc[0])

df_num

In [ ]:
# normaliza os dados numéricos
df_num = df_num.select_dtypes(include=['float64','int64'])
df_num = pd.DataFrame(scaler.fit_transform(df_num), columns=df_num.columns)


# normaliza os dados categóricos
df_cat = df_cat[['Kingdom']]
df_cat = pd.get_dummies(df_cat)

# concatena as duas partes do dataset
dataframe = pd.concat([df_num, df_cat], axis=1)
# imprime o dataset final
dataframe

In [ ]:
distorcoes = []
K = range(1,101)
for k in K: 
  modelo = KMeans(n_clusters=k,random_state=42,n_init='auto').fit(dataframe)
  distorcoes.append(sum(np.min(cdist(dataframe, modelo.cluster_centers_, 'euclidean'), axis=1)) / dataframe.shape[0])
  
plt.plot(distorcoes)

In [ ]:
import math
x0 = K[0]
y0 = distorcoes[0]

x1 = K[len(K)- 1]
y1 = distorcoes[len(distorcoes)-1]

distancias = []
for i in range(len(distorcoes)):
  x = K[i]
  y = distorcoes[i]
  numerador = abs((y1-y0)*x - (x1-x0)*y + x1*y0 - y1*x0)
  denominador = math.sqrt((y1-y0)**2 + (x1-x0)**2)
  distancias.append(numerador/denominador)

n_clusters_otimo = K[distancias.index(np.max(distancias))]
print('Numero ideal de clusters:', K[distancias.index(np.max(distancias))])

In [ ]:
#Treinar o modelo definitivo
from pickle import dump
modelo = KMeans(n_clusters=n_clusters_otimo,random_state=42,n_init='auto').fit(dataframe)

dump(modelo,open('cluster1.modelo','wb'))

In [ ]:
dados_num_desnormalizados = modelo_normalizador.inverse_transform(dados_num_normalizado)
dados_num_desnormalizados_DF = pd.DataFrame(dados_num_desnormalizados, columns= df_num.columns)
dados_num_desnormalizados_DF.head()

In [ ]:
# dados_cat_normalizado
df_cat.columns[0].split('_')[1]

In [ ]:
dados_num_desnormalizados_DF['Sexo']= ''
dados_num_desnormalizados_DF['Sexo'] = df_cat.apply(lambda row: 'F' if row['sexo_F'] > row['sexo_M'] else 'M', axis=1)
dados_num_desnormalizados_DF